In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

### Feature Store

In [2]:
selected_features = [
    'music name', 
    'music id',
    'acousticness',
    'danceability',
    'energy',
    'instrumentalness',
    'liveness',
    'loudness',
    'speechiness',
    'tempo',
    'valence',
    'duration_ms',
    'key',
    'mode',
    'time_signature'
]

### Authentication to Spotify API

In [3]:
with open('credentials.json', 'r') as file:
    credentials = json.load(file)
# 特徴量を格納するための空のリスト
features_list = []

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=credentials['client_id'],
                                               client_secret=credentials['client_secret'],
                                               redirect_uri=credentials['redirect_uri'],
                                               scope='user-read-recently-played'))

### Set Playlist ID

In [4]:
## ②プレイリスト
playlist_id = credentials['ops_playlist_id']

### Storing Feature Param

In [5]:
offset = 0
while True:
    tracks = sp.playlist_tracks(playlist_id, offset=offset, fields='items.track.id,items.track.name,total')
    if len(tracks['items']) == 0:
        break

    for track in tracks['items']:
        track_id = track['track']['id']
        track_name = track['track']['name']

        # 各トラックIDに対して特徴量を取得
        features = sp.audio_features(track_id)[0]
        features_list.append([track_name,track_id,features])
        # features_list.append(features) 

    offset += len(tracks['items'])

In [6]:
df = pd.DataFrame(features_list, columns=['music name', 'music id', 'features'])

# 特徴量の辞書を展開して新たなカラムを作成
features_df = df['features'].apply(pd.Series)

# 元のDataFrameと特徴量のDataFrameを結合
df = pd.concat([df[['music name', 'music id']], features_df], axis=1)
selected_features_df=df[selected_features]

In [7]:
# CSV保存
selected_features_df.to_csv("./test/opsdata.csv", index=False)

In [8]:
selected_features_df.tail()


,music name,music id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_ms,key,mode,time_signature
177,FLARE,18dWBsEdWdGA1MRPN95H1X,0.188000,0.413,0.915,0.000000,0.5260,-4.741,0.1250,164.008,0.465,274427,7,0,4
178,Daylight,7gDHjM9dOawzT6ErD08MPO,0.024300,0.487,0.823,0.000000,0.0895,-8.725,0.0470,165.986,0.694,239544,11,1,4
179,Blow out,7FAmTYG5KKZXkAk6fzlh7Y,0.000223,0.484,0.940,0.000000,0.3080,-2.581,0.0938,96.060,0.434,217880,6,0,4
180,TIT FOR TAT,19HUwJcc2hh2ss83LopTef,0.000092,0.399,0.949,0.000021,0.5480,-5.622,0.1530,172.673,0.559,242187,10,0,4
181,LIES GOES ON,2TXgyfocJAByylGfNv0xBy,0.017500,0.469,0.963,0.000000,0.3820,-2.431,0.1300,172.090,0.497,197433,10,0,4
